In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

df = pd.read_csv('clean_data1.csv')
df = df.drop(['Resume'],axis=1)
df.rename(columns={'newer_res':'Resume'},inplace=True)
df.head()

,Unnamed: 0,ID,Category,Resume
0,0,1,HR,"john h. smith , p.h.r . 800-991-5187 | po box ..."
1,1,2,HR,name surname address mobile no/email personal ...
2,2,3,HR,anthony brown hr assistant areas of expertise ...
3,3,4,HR,www.downloadmela.com satheesh email id : caree...
4,4,5,HR,human resources director expert organizational...


In [24]:
from io import StringIO
col = ['Category', 'Resume']
df = df[col]
df = df[pd.notnull(df['Resume'])]
df.columns = ['Category', 'Resume']
df['category_id'] = df['Category'].factorize()[0]
category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)

In [43]:
res_train, res_test, cat_train, cat_test=  train_test_split(df['Resume'], df['Category'], test_size=0.2, random_state=10)

In [44]:
vectorizer= CountVectorizer()
res_counts= vectorizer.fit_transform(res_train)

tfidf= TfidfTransformer()
res_tfidf= tfidf.fit_transform(res_counts)


In [45]:
from sklearn.svm import SVC
classifier= SVC(kernel='linear')
classifier.fit(res_counts, cat_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [46]:
predicted = []
for i in res_test:
    predicted.append((classifier.predict(vectorizer.transform([i])))[0])

In [57]:
a= pd.DataFrame(cat_test)
a['predicted'] = predicted
a.tail()

,Category,predicted
665,Consultant,Business Development
699,Digital Media,Digital Media
403,Advocate,Advocate
520,Health & Fitness,Finance
888,Engineering,Engineering


In [58]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
accuracy=accuracy_score(a.Category, a.predicted)
print("Accuracy from SVM:",accuracy)   

Accuracy from SVM: 0.7
